# Training SSD300 (tutorial)
In this tutorial, we will show how to initialize, train and test SSD300 for object detection.
The tutorial is based on a subset of pictures from the PascalVOC dataset (both 2007 and 2012), but can be generalized to be used with the whole PascalVOC or other custom/benchmark datasets.

### Imports
We start by importing all the necessary functions.

In [1]:
import torch
from PIL import Image
from time import time
import numpy as np
import matplotlib.pyplot as plt
import os
import torchvision.transforms as transforms

import sys
sys.path.insert(0, '/scratch/lmeneghe/Smithers/')

from smithers.ml.models.vgg import VGG
from smithers.ml.models.aux_conv import AuxiliaryConvolutions
from smithers.ml.models.predictor import PredictionConvolutions
from smithers.ml.dataset.pascalvoc_dataset import PascalVOCDataset
from smithers.ml.models.detector import Detector
from smithers.ml.models.utils_objdet import create_prior_boxes


import warnings
warnings.filterwarnings("ignore")

### Parameters Initialization
We set the parameters used for the data, the detector and the learning phase.

In [2]:
# Learning parameters
batch_size = 8  # batch size
workers = 4  # number of workers for loading data in the DataLoader
iterations = 120000  # number of iterations to train
print_freq = 200  # print training status every __ batches
lr = 1e-4  # learning rate
decay_lr_at = [80000, 100000]  # decay learning rate after these many iterations
decay_lr_to = 0.1
# decay learning rate to this fraction of the existing learning rate
#n_classes = 6
momentum = 0.9  # momentum
weight_decay = 5e-4  # weight decay
grad_clip = None
# clip if gradients are exploding, which may happen at larger batch sizes

### Dataset Loading
We need to load and create the datasets for training and testing our model. In this case we are using a dataset extracted from PascalVOC that has been created using ***smithers/ml/dataset/sample_dataset.py***. For more details about this, refer to the tutorial ***pascalvoc_preparation***, which explains also how to use the whole PascalVOC dataset. Instead, to use a custom dataset for this purpose, refer to the turial ***customdata_objdet***.

In [3]:
#voc_labels = ('aeroplane', 'bicycle', 'bird', 'boat',
#        'bottle', 'bus', 'car', 'cat', 'chair',
#        'cow', 'diningtable', 'dog', 'horse',
#        'motorbike', 'person', 'pottedplant',
#        'sheep', 'sofa', 'train', 'tvmonitor')
voc_labels = ('cat', 'dog')
label_map = {k: v + 1 for v, k in enumerate(voc_labels)}
label_map['background'] = 0
n_classes = len(label_map)
print('categories:',label_map)
print('n_classes:', n_classes)

# Data parameters
data_folder = 'VOC_dog_cat/JSONfiles' #folder with json data files
keep_difficult = True


train_dataset = PascalVOCDataset(data_folder,
                                 split='train',
                                 keep_difficult=keep_difficult)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=train_dataset.collate_fn,
    num_workers=workers,
    pin_memory=True)

epochs = iterations // (len(train_dataset) // 16) #500
decay_lr_at = [it // (len(train_dataset) // 16) for it in decay_lr_at]
print('Training images:', len(train_dataset))


test_dataset = PascalVOCDataset(data_folder,
                                split='test',
                                keep_difficult=keep_difficult)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          collate_fn=test_dataset.collate_fn,
                                          num_workers=workers,
                                          pin_memory=True)
print('Testing images:', len(test_dataset))


categories: {'cat': 1, 'dog': 2, 'background': 0}
n_classes: 3
Training images: 240
Testing images: 60


### Creation and loading of the SSD300 model
We are now going to initialize SSD300 using the class ***Detector*** of ***smithers/ml/models/detector.py***, based on the original paper: 

'SSD: Single Shot Multibox Detector' by Wei Liu, Dragomir Anguelov, Dumitru Erhan, Christian Szegedy, Scott Reed, Cheng-Yang Fu, Alexander C. Berg https://arxiv.org/abs/1512.02325 ,DOI: 10.1007/978-3-319-46448-0_2

In this case, we should select 'ssd' as classifier for VGG in order to substitute the feedforward classification layers of the original VGG with convolutional layers. We have then initialized SSD300 using pre-trained wegiths on ImageNet (a common choice in this field).


In [4]:
start_init = time()
base_net = VGG(classifier='ssd', init_weights='imagenet')
aux_conv = AuxiliaryConvolutions()
predictor = PredictionConvolutions(n_classes)
network = [base_net, aux_conv, predictor]
priors_cxcy = create_prior_boxes()
end_init = time()
print('Time needed to initialize the net: {} seconds'.format(round(end_init-start_init,2)))


Loaded base model.

Time needed to initialize the net: 2.57 seconds


### Training Phase
Once we have created and initliazed the network we can train it.


Here, the network will trained and, if provided, a checkpoint can be used to start from an already trained network.

The training procedure will store a checkpoint file named ``checkpoint_ssd300.pth.tar``, once it is completed.
Furthermore, the plot of the value of the loss against the corresponding epoch is produced.

In [ ]:
check = None # if no checkpoint available
#check = 'checkpoint_ssd300.pth.tar' #if a checkpoint is available
epochs = 15
start = time()
detector = Detector(network, check, priors_cxcy, n_classes, epochs,
                        batch_size, print_freq, lr, decay_lr_at,
                        decay_lr_to, momentum, weight_decay, grad_clip,
                        train_loader, test_loader)
print(detector.model)
checkpoint, loss_values, mAP_values = detector.train_detector_with_eval(label_map)
end = time()
print(f'Time needed for training the net: {round(end-start,2)} seconds, i.e. {round((end-start)/60,1)} minutes')
start_test = time()
epo = np.arange(start=0, stop=epochs, step=1)
plt.plot(epo, loss_values)
plt.xlabel('Epochs')
plt.ylabel('Value Loss')
plt.savefig('loss_pascal_cat.png')
detector.eval_detector(label_map, checkpoint)
end_test = time()

print(f'Time needed for testing the net: {round(end_test-start_test,2)} seconds, i.e. {round((end_test-start_test)/60,1)} minutes')

### TESTING WITH AN IMAGE + PRINT
By choosing a specific image from the directory used, the network is able to detect the object(s) in it and save a version of the image in which the bounding box(es) with the relative label(s) is(/are) pictured. To view this, open the file ``out.jpg``.

In [ ]:
img_path = '/u/s/szanin/Smithers/smithers/ml/tutorials/VOC_dog_cat/JPEGImages/001239.jpg'
#img_path = 'voc_dir/VOC_cat-dog/JPEGImages/001239.jpg'
original_image = Image.open(img_path, mode='r')
original_image = original_image.convert('RGB')

detector.detect(original_image,
                checkpoint,
                label_map,
                min_score=0.01,
                max_overlap=0.45,
                top_k=3).show()